In [2]:
source "${HOME}/conda/etc/profile.d/conda.sh"

Activation qiime2

In [3]:
conda activate qiime2-2023.7

(qiime2-2023.7) 


: 1

Make sure you're in the correct directory

In [4]:
cd /work/OVERFLOW/PathScreen/Antifungal_Isolates

(qiime2-2023.7) 


: 1

In [5]:
pwd

/work/OVERFLOW/PathScreen/Antifungal_Isolates
(qiime2-2023.7) 


: 1

Need to get only the caecilian skin reads from the dataset

In [29]:
cp Caecilian_Micro_Metadata_2020.2.txt Caecilian_Micro_Metadata_2020.2.tsv

(qiime2-2023.7) 


: 1

In [30]:
qiime feature-table filter-samples \
  --i-table table-dada2-caecilian-paired-reads.qza \
  --m-metadata-file Caecilian_Micro_Metadata_2020.2.tsv \
  --p-where "[Category]='caecilian'" \
  --o-filtered-table caecilian-filtered-table.qza

Saved FeatureTable[Frequency] to: caecilian-filtered-table.qza
(qiime2-2023.7) 


: 1

In [31]:
qiime taxa filter-seqs \
  --i-sequences rep-seqs-caecilian-paired.qza \
  --i-table caecilian-filtered-table.qza \
  --o-filtered-sequences rep-seqs-caecilian-only.qza

Usage: qiime taxa filter-seqs [OPTIONS]

  This method filters sequences based on their taxonomic annotations. Features
  can be retained in the result by specifying one or more include search
  terms, and can be filtered out of the result by specifying one or more
  exclude search terms. If both include and exclude are provided, the
  inclusion critera will be applied before the exclusion critera. Either
  include or exclude terms (or both) must be provided.

Inputs:
  --i-sequences ARTIFACT FeatureData[Sequence]
                          Feature sequences to be filtered.         [required]
  --i-taxonomy ARTIFACT FeatureData[Taxonomy]
                          Taxonomic annotations for features in the provided
                          feature sequences. All features in the feature
                          sequences must have a corresponding taxonomic
                          annotation. Taxonomic annotations for features that
                          are not present in the featur

: 1

Didn't work to use the table to filter the sequences, just going to use the whole dataset

Import sequences corresponding to the Antifungal Isolates Database

Note: To import a .fna file, I followed this tutorial (https://docs.qiime2.org/2024.5/tutorials/otu-clustering/) linked from the qiime2 importing data tutorial. These file types are sequences without quality information and type must be specific to 'SampleData' for this import to work.

In [14]:
qiime tools import --input-path Amphibianskin_bacteria_16S_sequences.fna --output-path Amphibianskin_bacteria_16S_sequences.qza --type 'SampleData[Sequences]'

Imported Amphibianskin_bacteria_16S_sequences.fna as QIIME1DemuxDirFmt to Amphibianskin_bacteria_16S_sequences.qza
(qiime2-2023.7) 


: 1

Have to run the code below in order to get sequence file in the correct format for downstream use

Info from this tutorial (https://docs.qiime2.org/2024.5/tutorials/otu-clustering/#dereplicating-a-sampledata-sequences-artifact). 
"The outputs from dereplicate-sequences are a FeatureTable[Frequency] artifact and a FeatureData[Sequence] artifact. The FeatureTable[Frequency] artifact is the feature table indicating the number of times each amplicon sequence variant (ASV) is observed in each of your samples. The FeatureData[Sequence] contains the mapping of each feature identifier to the sequence variant that defines that feature. These files are analogous to those generated by qiime dada2 denoise-* and qiime deblur denoise-*, except that no denoising, chimera removal, or other quality control has been applied in the dereplication process. (In this example, the only quality control of these data is what was applied outside of QIIME 2, before the import step.)"

In [15]:
qiime vsearch dereplicate-sequences \
  --i-sequences Amphibianskin_bacteria_16S_sequences.qza \
  --o-dereplicated-table table_Amphibianskin_bacteria_16S_sequences.qza \
  --o-dereplicated-sequences rep-seqs_Amphibianskin_bacteria.qza

Saved FeatureTable[Frequency] to: table_Amphibianskin_bacteria_16S_sequences.qza
Saved FeatureData[Sequence] to: rep-seqs_Amphibianskin_bacteria.qza
(qiime2-2023.7) 


: 1

Used the code below to cluster sequences

In [16]:
qiime vsearch cluster-features-closed-reference \
  --i-table table-dada2-caecilian-paired-reads.qza \
  --i-sequences rep-seqs-caecilian-paired.qza \
  --i-reference-sequences rep-seqs_Amphibianskin_bacteria.qza \
  --p-perc-identity 1 \
  --o-clustered-table table-cr-100.qza \
  --o-clustered-sequences rep-seqs-cr-100.qza \
  --o-unmatched-sequences unmatched-cr-100.qza

Saved FeatureTable[Frequency] to: table-cr-100.qza
Saved FeatureData[Sequence] to: rep-seqs-cr-100.qza
Saved FeatureData[Sequence] to: unmatched-cr-100.qza
(qiime2-2023.7) 


: 1

In [ ]:
qiime feature-table summarize \
  --i-table table-cr-100.qza \
  --o-visualization table-cr-100.qzv

From (https://docs.qiime2.org/2024.5/tutorials/otu-clustering/#dereplicating-a-sampledata-sequences-artifact). "The outputs from cluster-features-closed-reference are a FeatureTable[Frequency] artifact and two FeatureData[Sequence] artifacts. The FeatureData[Sequence] artifacts consist of the sequences defining the features in the FeatureTable (rep-seqs-cr-85.qza from the command block above) as well as the collection of feature ids and their sequences that didn’t match the reference database at 85% identity. The reference sequences provided as input should be used as sequences defining the features in the FeatureTable in closed-reference OTU picking."

In [17]:
qiime feature-table tabulate-seqs \
  --i-data rep-seqs-cr-100.qza \
  --o-visualization rep-seqs-cr-100.qzv

Saved Visualization to: rep-seqs-cr-100.qzv
(qiime2-2023.7) 


: 1

Took a look at the visualization, got 78 matches to the reference sequences (i.e., 78 sequences from our caecilian skin swabs matched at 100% identity to the Antifungal Isolates Database)

Imported the taxonomy to try and see what taxa these sequences represent

In [7]:
qiime tools import --type 'FeatureData[Taxonomy]' --input-format HeaderlessTSVTaxonomyFormat --input-path Antifungal_Isolates_Database_Taxonomy.txt --output-path Antifungal_Isolates_Database_Taxonomy.qza

Imported Antifungal_Isolates_Database_Taxonomy.txt as HeaderlessTSVTaxonomyFormat to Antifungal_Isolates_Database_Taxonomy.qza
(qiime2-2023.7) 


: 1

See code I tried below. Did not work because the sequencing file and information I have from downloading the Antifungal Isolates Database doesn't match up with what is required for formatting in qiime2; can't seem to find a way to match the seqs output with assigned taxonomy according to what is anti-Bd bacteria, etc.

In [23]:
qiime metadata tabulate \
--m-input-file Antifungal_Isolates_Database_Taxonomy.qza \
--m-input-file rep-seqs-cr-100.qza \
--o-visualization merged-table.qzv

There was an issue with merging QIIME 2 Metadata:

  Cannot merge because there are no IDs shared across metadata objects.

(qiime2-2023.7) 


: 1

In [22]:
qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads rep-seqs_Amphibianskin_bacteria.qza \
  --i-reference-taxonomy Antifungal_Isolates_Database_Taxonomy.qza \
  --o-classifier classifier.qza

Plugin error from feature-classifier:

  not enough values to unpack (expected 2, got 0)

Debug info has been saved to /tmp/qiime2-q2cli-err-v47n7r8t.log
(qiime2-2023.7) 


: 1

Found a tutorial (https://docs.qiime2.org/2017.12/tutorials/quality-control/#excluding-sequences-by-alignment) for aligning seqs using vsearch methods and getting outputs of hits and misses. Here the hits are what we want, these are seqs from our caecilian skin swabs that aligned and matched at 100% identity to the Antifungal Isolates Database

In [24]:
qiime quality-control exclude-seqs \
  --i-query-sequences rep-seqs-caecilian-paired.qza \
  --i-reference-sequences rep-seqs_Amphibianskin_bacteria.qza \
  --p-method vsearch \
  --p-perc-identity 1 \
  --p-perc-query-aligned 1 \
  --o-sequence-hits hits.qza \
  --o-sequence-misses misses.qza

Saved FeatureData[Sequence] to: hits.qza
Saved FeatureData[Sequence] to: misses.qza
(qiime2-2023.7) 


: 1

In [25]:
qiime feature-table tabulate-seqs \
  --i-data hits.qza \
  --o-visualization hits.qzv

Saved Visualization to: hits.qzv
(qiime2-2023.7) 


: 1

This output shows 113 matching sequences

I downloaded a raw FASTA file from the vizualization page (view.qiime2.org) of the hits.qzv file

Maria sent the results of the DESeq2 analysis and thought it might be usefult to compare those sequences outputs against the Antifungal Isolates Database to see if bacteria more abundant on caecilian skin is relatively more antifungal, so made a txt file with the feature IDs of those 160 ASVs and filtered the caecilian sequences to only these most abundant taxa.

In [7]:
qiime feature-table filter-seqs \
     --i-data rep-seqs-caecilian-paired.qza \
     --m-metadata-file DESeq2_output_seqs.txt \
     --o-filtered-data seqs-filt.qza

Saved FeatureData[Sequence] to: seqs-filt.qza
(qiime2-2023.7) 


: 1

Make sure they look like what I expect

In [8]:
qiime feature-table tabulate-seqs \
  --i-data seqs-filt.qza \
  --o-visualization seqs-filt.qzv

Saved Visualization to: seqs-filt.qzv
(qiime2-2023.7) 


: 1

Use the same exclude-seqs function to get a table of ASVs that match the Antifungal Isolates database at 100% sequence identity

In [9]:
qiime quality-control exclude-seqs \
  --i-query-sequences seqs-filt.qza \
  --i-reference-sequences rep-seqs_Amphibianskin_bacteria.qza \
  --p-method vsearch \
  --p-perc-identity 1 \
  --p-perc-query-aligned 1 \
  --o-sequence-hits DESeq2-hits.qza \
  --o-sequence-misses DESeq2-misses.qza

Saved FeatureData[Sequence] to: DESeq2-hits.qza
Saved FeatureData[Sequence] to: DESeq2-misses.qza
(qiime2-2023.7) 


: 1

In [10]:
qiime feature-table tabulate-seqs \
  --i-data DESeq2-hits.qza \
  --o-visualization DESeq2-hits.qzv

Saved Visualization to: DESeq2-hits.qzv
(qiime2-2023.7) 


: 1

Visualized the table and saved the resulting .fasta file

Filtering the misses information from the dataset to try and see the feature table for the hits data

In [26]:
qiime feature-table filter-features \
  --i-table table_Amphibianskin_bacteria_16S_sequences.qza \
  --m-metadata-file misses.qza \
  --o-filtered-table only-hits-filtered-table.qza

Saved FeatureTable[Frequency] to: only-hits-filtered-table.qza
(qiime2-2023.7) 


: 1

In [27]:
qiime feature-table summarize \
  --i-table only-hits-filtered-table.qza \
  --o-visualization only-hits-filtered-table.qzv

Saved Visualization to: only-hits-filtered-table.qzv
(qiime2-2023.7) 


: 1

Tried to look at the feature table for this information but came back blank, I think table input is wrong but can't fix it since I didn't process the raw sequencing data in qiime2 from scratch, just using what I got from the data download

Lastly, trying this code to see if it gives me anything different

In [28]:
qiime feature-classifier classify-consensus-vsearch --i-query rep-seqs-caecilian-paired.qza --i-reference-reads rep-seqs_Amphibianskin_bacteria.qza --i-reference-taxonomy Antifungal_Isolates_Database_Taxonomy.qza --p-perc-identity 1.0 --o-classification vsearch-isolates-assign-100.qza --o-search-results isolates-search-res-100.qza --verbose

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --usearch_global /tmp/qiime2/smcgrath/data/09d73a55-d379-4658-a6b9-2dac06ea415b/data/dna-sequences.fasta --id 1.0 --query_cov 0.8 --strand both --maxaccepts 10 --maxrejects 0 --db /tmp/qiime2/smcgrath/data/eb878246-9a1c-4ad5-8884-775b3be7a158/data/dna-sequences.fasta --threads 1 --output_no_hits --blast6out /tmp/q2-BLAST6Format-sjhqw3r7

vsearch v2.22.1_linux_x86_64, 376.4GB RAM, 48 cores
https://github.com/torognes/vsearch

Reading file /tmp/qiime2/smcgrath/data/eb878246-9a1c-4ad5-8884-775b3be7a158/data/dna-sequences.fasta 100%  
2136187 nt in 1913 seqs, min 98, max 1488, avg 1117
Masking 100% 
Counting k-mers 100% 
Creating k-mer index 100% 
Searching 100%  
Matching unique query sequences: 86 of 3064 (2.81%)
Traceback (most recent call last):
  Fi

: 1

Got an error saying the reference taxonomy and search results don't match, think this is because it only gives me a sequence ID value that doesn't correspond to anything in the other files. Oh well